In [1]:
#### t_fwd3_t vs t_fwd3 on train=False (we can't do this testing for train=True due to different RNGs tooling)
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_transformer_gpt2
BS, H, N, D = 2, 2, 512, 64 # 1, 1, 3, 4 #2, 2, 5, 4
vocab_size = 128
layers = 2
p_gen_aux = [42] + [43,44,45] * layers
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N+1), device="cuda").to(torch.int32)
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
train = False

for i, i_mask in enumerate(mask):
    mask[i] = torch.tril(i_mask)
    #mask[i] = torch.zeros_like(i_mask)
# print(mask)

from model_triton import t_gpt2_forward_with_acts, t_gpt2_forward_with_acts_t
y_in = y[:, :-1]
y_out = y[:, 1:]
    
res2, acts2 = t_gpt2_forward_with_acts(layers_params, y_in, mask, None, train, p_gen_aux)
print(res2)
#print(res2[1])
#print_res_shapes(res2[0]) 

print(f'----X----')

res3, acts3 = t_gpt2_forward_with_acts_t(layers_params, y_in, mask, None, train, p_gen_aux)
print(res3)
#print_res_shapes(res3[0]) 

assert torch.allclose(res2, res3, rtol=1e-3, atol=2e-4)

print(f'----XXX----')

def recursive_assert(a, b):
    if isinstance(a, torch.Tensor):
        assert isinstance(b, torch.Tensor)
        torch.allclose(a, b, rtol=1e-3, atol=1e-4)
    else:
        assert not isinstance(b, torch.Tensor)
        if len(a) != len(b):
            return
        assert len(a) == len(b), f'len(a) {len(a)}, len(b) {len(b)}'
        for i, (ai, bi) in enumerate(zip(a, b)):
            recursive_assert(ai, bi)
recursive_assert(acts2, acts3)
print(f'----XXX----')




tensor([[[-0.121750, -0.049716,  0.255192,  ..., -0.095632,  0.082414,
          -0.118754],
         [-0.156099, -0.205412,  0.323758,  ...,  0.193226,  0.046833,
           0.331142],
         [-0.163206, -0.231650,  0.123777,  ..., -0.023117, -0.302415,
          -0.250485],
         ...,
         [ 0.111507, -0.186753,  0.164638,  ..., -0.016895, -0.142807,
           0.094000],
         [-0.061341,  0.086395,  0.189143,  ...,  0.041986,  0.263310,
          -0.111488],
         [ 0.171672, -0.020613, -0.329271,  ...,  0.219474,  0.067053,
           0.396186]],

        [[-0.007150, -0.051896, -0.020035,  ...,  0.179652,  0.125414,
          -0.117793],
         [ 0.012357, -0.069802, -0.234931,  ...,  0.080076, -0.267802,
           0.081603],
         [-0.095608,  0.052709, -0.296614,  ...,  0.089055,  0.044480,
           0.055714],
         ...,
         [-0.040908,  0.090995, -0.020188,  ..., -0.188718,  0.172418,
          -0.065628],
         [-0.100947,  0.079405, -0.07757

loc(callsite("/efs/notebooks/mkukla/pre-tjax/model_triton.py":1463:7 at "/efs/notebooks/mkukla/pre-tjax/model_triton.py":690:64)): error: operation scheduled before its operands
loc(callsite("/efs/notebooks/mkukla/pre-tjax/model_triton.py":1463:7 at "/efs/notebooks/mkukla/pre-tjax/model_triton.py":690:64)): error: operation scheduled before its operands
loc(callsite("/efs/notebooks/mkukla/pre-tjax/model_triton.py":1463:7 at "/efs/notebooks/mkukla/pre-tjax/model_triton.py":690:64)): error: operation scheduled before its operands
loc(callsite("/efs/notebooks/mkukla/pre-tjax/model_triton.py":1463:7 at "/efs/notebooks/mkukla/pre-tjax/model_triton.py":690:64)): error: operation scheduled before its operands


tensor([[[-0.121750, -0.049716,  0.255192,  ..., -0.095632,  0.082414,
          -0.118754],
         [-0.156089, -0.205429,  0.323747,  ...,  0.193217,  0.046856,
           0.331167],
         [-0.163225, -0.231646,  0.123783,  ..., -0.023115, -0.302414,
          -0.250477],
         ...,
         [ 0.111507, -0.186753,  0.164638,  ..., -0.016895, -0.142807,
           0.094000],
         [-0.061351,  0.086386,  0.189182,  ...,  0.041987,  0.263344,
          -0.111471],
         [ 0.171679, -0.020618, -0.329262,  ...,  0.219468,  0.067055,
           0.396186]],

        [[ 0.008889, -0.048972, -0.003976,  ...,  0.196161,  0.116899,
          -0.128681],
         [ 0.023490, -0.075754, -0.222352,  ...,  0.093987, -0.273189,
           0.076449],
         [-0.090338,  0.042223, -0.286589,  ...,  0.090479,  0.053619,
           0.045792],
         ...,
         [-0.041011,  0.090959, -0.020124,  ..., -0.188746,  0.172540,
          -0.065610],
         [-0.100985,  0.079360, -0.07774

AssertionError: 